In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 1. Combine Lamas 2018 report with WGS coords


In Notebook 1 I took the Israely Central Statistic Bureau (Lamas) report. 
I extracted all cities (city, kibutz, moshav, village ext.) and got their coordinates using the Gmaps API.
Here we will paste the coords back to the Lamas table - to be used later on for Info-Geo visualization.

In [2]:
#So now after I got the WGS for each city I want to reload the original Lamas file, and add the WGS coordinates to eact city
file = r'..\csv\lamas_report_2018.xlsx'
lamas = pd.read_excel(file)                   


In [3]:
#lets change the columns heads to english
#generate new english column titles
engColumns = ['nameHeb', 'code', 'pronaunce', 'district', 'shire', 'naturalArea', 'monicipal',
             'metro', 'religion', 'population', 'jewsAndOthers', 'jews', 'arabs','estbYear', 'type', 'ITM', 'altitude',
             'orgazinationRelated', 'plansComit', 'policeArea', 'year', 'nameEng', 'localCuncil']
#assign the english col titles
lamas.columns  = engColumns

In [4]:
#we will set the city code as index
lamas.set_index('code', inplace = True)

In [5]:
#Now, lets load the CSV file with all israel cities WGS points we genereated in the previes notebook
#we will keep the original index from the LAMAS file, just in case
file = r'..\csv\israel_cities_with_lamas_codes_wgs.csv'
israel_wgs = pd.read_csv(file, usecols = [0, 1, 3, 4], index_col = 0)
israel_wgs.head()


,name,lat,long
code,,,
967,Abu Juway'ad,NaN,NaN
472,Abu Ghosh,31.806293,35.109193
473,Abu Sinan,32.958030,35.171969
935,Abu Surayhan,NaN,NaN
958,Abu 'Abdun,31.301397,34.841655


In [6]:
#and now lets merge the two dfs based on the city code
lamas_wgs = pd.merge(lamas, israel_wgs, left_index = True, right_index = True)

In [7]:
lamas_wgs.head()

,nameHeb,pronaunce,district,shire,naturalArea,monicipal,metro,religion,population,jewsAndOthers,...,altitude,orgazinationRelated,plansComit,policeArea,year,nameEng,localCuncil,name,lat,long
code,,,,,,,,,,,,,,,,,,,,,
967,אבו ג'ווייעד (שבט),ABU JUWEI'ID,6,62,623.0,NaN,NaN,3.0,NaN,NaN,...,2.040057e+09,NaN,699.0,15003711.0,2018,Abu Juway'ad,NaN,Abu Juway'ad,NaN,NaN
472,אבו גוש,ABU GHOSH,1,11,111.0,99.0,444.0,2.0,7543.0,97.0,...,2.105263e+09,598.0,152.0,10002475.0,2018,Abu Ghosh,NaN,Abu Ghosh,31.806293,35.109193
473,אבו סנאן,ABU SINAN,2,24,245.0,99.0,NaN,2.0,13915.0,28.0,...,2.160776e+09,19.0,252.0,10004315.0,2018,Abu Sinan,NaN,Abu Sinan,32.958030,35.171969
935,אבו סריחאן (שבט),ABU SUREIHAN,6,62,623.0,NaN,NaN,3.0,NaN,NaN,...,1.865057e+09,NaN,699.0,10001937.0,2018,Abu Surayhan,NaN,Abu Surayhan,NaN,NaN
958,אבו עבדון (שבט),ABU ABDUN,6,62,623.0,NaN,NaN,3.0,NaN,NaN,...,1.850058e+09,NaN,699.0,10001937.0,2018,Abu 'Abdun,NaN,Abu 'Abdun,31.301397,34.841655


# Check the data and clean weird WGS points

In [8]:
#So, there are many columns. To easly slice them by index we can show them with thier column-index:
pd.Series(lamas_wgs.columns)

0                 nameHeb
1               pronaunce
2                district
3                   shire
4             naturalArea
5               monicipal
6                   metro
7                religion
8              population
9           jewsAndOthers
10                   jews
11                  arabs
12               estbYear
13                   type
14                    ITM
15               altitude
16    orgazinationRelated
17             plansComit
18             policeArea
19                   year
20                nameEng
21            localCuncil
22                   name
23                    lat
24                   long
dtype: object

In [9]:
#Now lets slice features of intrest and check the WGS coordinates
israel = lamas_wgs[lamas_wgs.columns[[22, 8, 23, 24]]]
israel.head()


,name,population,lat,long
code,,,,
967,Abu Juway'ad,NaN,NaN,NaN
472,Abu Ghosh,7543.0,31.806293,35.109193
473,Abu Sinan,13915.0,32.958030,35.171969
935,Abu Surayhan,NaN,NaN,NaN
958,Abu 'Abdun,NaN,31.301397,34.841655


In [10]:
#let's check our missing values:
israel[israel.isna().any(axis=1)]

,name,population,lat,long
code,,,,
967,Abu Juway'ad,NaN,NaN,NaN
935,Abu Surayhan,NaN,NaN,NaN
958,Abu 'Abdun,NaN,31.301397,34.841655
1042,Abu 'Ammar,NaN,NaN,NaN
932,Abu 'Amra,NaN,31.282188,34.872739
968,Abu Kuraynat,NaN,NaN,NaN
966,Abu Rubay'a,NaN,NaN,NaN
961,Abu Rukayyek,NaN,NaN,NaN
965,Atrash,NaN,31.264619,34.948587


In [11]:
#our missing raws are either very small rural settelments or army bases ('Mahane') 
#for the sake of general analysis we can drop them 
israel.dropna(inplace = True)


In [12]:
#convert the coordinates to floats
israel['lat'] = israel['lat'].apply(float)
israel['long'] = israel['long'].apply(float)


In [13]:
#now lets examine our points. Remind you that 'lat' is messuring NORTH from the equaiotr and 'long' is messuring EAST of Greenwich (England)
print('northest city in israel is', israel['name'][israel['lat'].idxmax()])
print('southest city in israel is', israel['name'][israel['lat'].idxmin()])
print('eastest city in israel is', israel['name'][israel['long'].idxmax()])
print('wesenest city in israel is', israel['name'][israel['long'].idxmin()])



northest city in israel is Metula
southest city in israel is Elat
eastest city in israel is Allone HaBashan
wesenest city in israel is Dan


So, if you have a little geographic knowledge of Israel you  would know that these points make sense, except from Dan,which is a kibbutz on the northern border, but not very western. So lets check that:

In [14]:

israel[israel['name'] == 'Dan']



,name,population,lat,long
code,,,,
303,Dan,732.0,33.239745,-35.653803


In [15]:
#This minus got there for some reason. so lets change that:
# israel.loc[303,:]['long'] = 35.653803
israel.at[303, 'long'] =  35.653803

#Let's re-check
print('wesenest city in israel is', israel['name'][israel['long'].idxmin()])



wesenest city in israel is Shlomit


Well, never heard of this place , but a quick google search shows it exist at the western tip of israel, next to the egyptian border
If you are still not sure, sample random raws and "goo-validate" thier position 

In [16]:
#And lets change back the full df also, and save it
lamas_wgs.at[303, 'long'] =  35.653803


In [17]:
#another check for weird vallues
lamas_wgs[lamas_wgs['lat'] < 29]

,nameHeb,pronaunce,district,shire,naturalArea,monicipal,metro,religion,population,jewsAndOthers,...,altitude,orgazinationRelated,plansComit,policeArea,year,nameEng,localCuncil,name,lat,long
code,,,,,,,,,,,,,,,,,,,,,


In [18]:
#another check for weird values
lamas_wgs[lamas_wgs['long'] < 29]

,nameHeb,pronaunce,district,shire,naturalArea,monicipal,metro,religion,population,jewsAndOthers,...,altitude,orgazinationRelated,plansComit,policeArea,year,nameEng,localCuncil,name,lat,long
code,,,,,,,,,,,,,,,,,,,,,
963,אעצם (שבט),A'SAM,6,62,623.0,NaN,NaN,3.0,NaN,NaN,...,1.910056e+09,NaN,699.0,15003711.0,2018,A'sam,NaN,A'sam,40.710248,-74.213615


In [19]:
lamas_wgs.drop(963, axis=0, inplace=True)

So overall - it looks good.
Now we can export the table for further analysis.
Of course, an easier to check the coordinates - is to visualize them on a map.
this is done in the next notebook.

note that I'm exporting the original table, that includes NA values.

In [20]:
#lamas_wgs.to_csv(r'..\lamas_report_2018_with_wgs.csv')